In [4]:
import torch
from torchcam.methods import GradCAM
from torchvision.models import resnet18
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torch.utils.data import DataLoader
from torchcam.utils import overlay_mask
from PIL import Image

import matplotlib.pyplot as plt


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:

# Step 1: Setup environment and load model
# Load pretrained model
model = resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 10)  # Adapt for CIFAR-10
model.load_state_dict(torch.load('cifar10_resnet18.pth'))
model.eval()

# Define transformation for input images
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Step 2: Generate and visualize explanations
# Initialize Grad-CAM
cam_extractor = GradCAM(model, 'layer4')

# Load and process sample image
img = Image.open("sample_airplane.jpg")
inputs = transform(img).unsqueeze(0)

# Generate heatmap
out = model(inputs)
activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)

# Overlay heatmap on the image
result = overlay_mask(img, activation_map[0], alpha=0.5)

# Display the result
plt.imshow(result)
plt.axis('off')
plt.show()

# Step 3: Play around with the layers
# You can change 'layer4' to other layers like 'layer3', 'layer2', etc.
# Observe the differences in the generated heatmaps.